In [1]:
from pprint import pprint
from bs4 import BeautifulSoup as bs
import requests
from time import sleep

## Спарсим данные

In [2]:
USER_AGENT = 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'

URL_FIRST_PAGE_HH = 'https://hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true'

headers = {
    'User-Agent': USER_AGENT,
}

In [3]:
def parse_hh(url_page, headers, result=[], index_page=1):
    response = requests.get(url_page, headers=headers)
    if response.status_code != 200:
      print('Парсинг завершен')
      return result
    else:
      print('Cтраница №%d, ссылка: %s'%(index_page, response.url))

    dom = bs(response.content, 'html.parser')
    vacancies = dom.find_all('div', {'class': 'vacancy-serp-item__layout'})
    for vacancy in vacancies:
      result.append(parse_vacancy_hh(vacancy))

    link_next_page = dom.find('a', {'data-qa': 'pager-next'})
    if link_next_page:
      link_next_page =  'https://spb.hh.ru' + link_next_page['href']
    else:
      print('Парсинг завершен')
      return result

    result = parse_hh(link_next_page, headers, result, index_page+1)   
    return result
     
def parse_vacancy_hh(dom_vacancy):
  sleep(0.1)
  vacancy_name = dom_vacancy.find('a').text

  vacancy_salary = dom_vacancy.find('span', {'class', 'bloko-header-section-3'})
  if vacancy_salary:
    vacancy_salary = vacancy_salary.text
    min_salary, max_salary, currency_salary = clean_salary(vacancy_salary)
  else:
    min_salary, max_salary, currency_salary = None, None, None

  vacancy_link = dom_vacancy.find('a')['href']
  
  return {
      'vacancy_name': vacancy_name,
      'vacancy_salary': vacancy_salary,
      'min_salary': min_salary,
      'max_salary': max_salary,
      'currency_salary': currency_salary,
      'vacancy_link': vacancy_link,
      'vacancy_source': 'hh.ru',
  }

def clean_salary(vacancy_salary_text, min_salary=None, max_salary=None, currency_salary=None):
  list_salary = vacancy_salary_text.replace('\u202f', '').split()
  for i in range(len(list_salary) - 1):
    if list_salary[i] == 'от':
      min_salary = int(list_salary[i + 1])
    elif list_salary[i] == 'до':
      max_salary = int(list_salary[i + 1])
    elif list_salary[i] == '–':
      min_salary = int(list_salary[i - 1])
      max_salary = int(list_salary[i + 1])
  currency_salary = list_salary[-1]

  return min_salary, max_salary, currency_salary

In [4]:
result = parse_hh(URL_FIRST_PAGE_HH, headers)

Cтраница №1, ссылка: https://hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true
Cтраница №2, ссылка: https://spb.hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&page=1&hhtmFrom=vacancy_search_list
Cтраница №3, ссылка: https://spb.hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&page=2&hhtmFrom=vacancy_search_list
Cтраница №4, ссылка: https://spb.hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&page=3&hhtmFrom=vacancy_search_list
Cтраница №5, ссылка: https://spb.hh.ru/search/vacancy?text=Java&from=suggest_post&salary=&area=88&no_magic=true&ored_clusters=true&items_on_page=20&enable_snippets=true&page=4&hhtmFrom=vacancy_search_list
Cтраница №6, сс

In [5]:
len(result)

123

## Запишем данные в базу

In [6]:
from pymongo import MongoClient

In [7]:
client = MongoClient()

In [8]:
db = client.vacancies_python_hh

In [9]:
collection_vacancies_hh_ru = db.hh_ru

In [10]:
def cheak_and_save_vacancies_in_db(vacancies):
  for vacancy in vacancies:
    if not len(list(collection_vacancies_hh_ru.find({'vacancy_link': vacancy['vacancy_link']}))):
      collection_vacancies_hh_ru.insert_one(vacancy)

In [11]:
cheak_and_save_vacancies_in_db(result)

In [12]:
result_find = list(collection_vacancies_hh_ru.find())
result_find

[{'_id': ObjectId('64102bdae6def747bf816c36'),
  'vacancy_name': 'Java Developer Trainee',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://hh.ru/vacancy/75529245?from=vacancy_search_list&query=Java',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64102bdae6def747bf816c37'),
  'vacancy_name': 'Разработчик Java/Jakarta (удаленно)',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://hh.ru/vacancy/77235716?from=vacancy_search_list&query=Java',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64102bdae6def747bf816c38'),
  'vacancy_name': 'Java-разработчик',
  'vacancy_salary': None,
  'min_salary': None,
  'max_salary': None,
  'currency_salary': None,
  'vacancy_link': 'https://hh.ru/vacancy/77775058?from=vacancy_search_list&query=Java',
  'vacancy_source': 'hh.ru'},
 {'_id': ObjectId('64102bdae6def747bf816c39'),
  'vacancy_name': 'Java Back

In [13]:
len(result_find)

123

In [14]:
def search_by_salary(collection, limit):
    vacancies = collection.find({'$or': [{'min_salary': {'$gt': limit}}, {'max_salary': {'$gt': limit}}]})
    for vacancy in vacancies:
        pprint(vacancy)

In [15]:
search_by_salary(collection_vacancies_hh_ru, 50000)

{'_id': ObjectId('64102bdae6def747bf816c3c'),
 'currency_salary': 'руб.',
 'max_salary': 450000,
 'min_salary': None,
 'vacancy_link': 'https://hh.ru/vacancy/77617915?from=vacancy_search_list&query=Java',
 'vacancy_name': 'Senior Java Backend',
 'vacancy_salary': 'до 450\u202f000 руб.',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('64102bdae6def747bf816c40'),
 'currency_salary': 'руб.',
 'max_salary': 450000,
 'min_salary': None,
 'vacancy_link': 'https://hh.ru/vacancy/77463926?from=vacancy_search_list&query=Java',
 'vacancy_name': 'Java-разработчик (удаленно/Сербия)',
 'vacancy_salary': 'до 450\u202f000 руб.',
 'vacancy_source': 'hh.ru'}
{'_id': ObjectId('64102bdae6def747bf816c46'),
 'currency_salary': 'руб.',
 'max_salary': None,
 'min_salary': 180000,
 'vacancy_link': 'https://hh.ru/vacancy/77330080?from=vacancy_search_list&query=Java',
 'vacancy_name': 'Реверс-инженер мобильных приложений в Дзен-мани',
 'vacancy_salary': 'от 180\u202f000 руб.',
 'vacancy_source': 'hh.ru'}
{'_id': O